<a href="https://colab.research.google.com/github/FaiqNasir525/Custom_CNN_Based_Image_Classification_Model/blob/main/CNN_Image_Classification_with_Advanced_Hyperparameter_Tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [ ]:
!pip install keras_tuner

import tensorflow as tf
from tensorflow.keras import datasets, layers, models, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from keras_tuner.tuners import RandomSearch
import matplotlib.pyplot as plt

# Loading and Preprocessing Data

In [ ]:
#Loading Data
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

#Normalizing from 0 to 1 Range
train_images, test_images = train_images / 255.0, test_images / 255.0

#Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
train_generator = datagen.flow(train_images, train_labels, batch_size=64)

#Writing class names fto better visulization
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Building Dynamic and Hyper-Tunable Model

In [ ]:
# Step 4: Define a function to build the model.
def build_model(hp):
    model = models.Sequential()

    # Tune the number of convolutional layers (1, 2 or 3)
    for i in range(hp.Int('conv_layers', 1, 3)):
        if i == 0:
            model.add(layers.Conv2D(
                filters=hp.Int('filters_' + str(i), min_value=32, max_value=128, step=16),
                kernel_size=3,
                activation='relu',
                kernel_regularizer=regularizers.l2(1e-4),
                padding='same',
                input_shape=(32, 32, 3)))
        else:
            for j in range(hp.Int('deep_conv_layers', 1, 3)):
                model.add(layers.Conv2D(
                    filters=hp.Int('filters_' + str(i) + str(j), min_value=64, max_value=128, step=16),
                    kernel_size=3,
                    activation='relu',
                    kernel_regularizer=regularizers.l2(1e-4),
                    padding='same'))
                model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(layers.Dropout(rate=hp.Float(f'dropoutConv_{i}', 0.0, 0.5, step=0.1)))

    model.add(layers.Flatten())

    # Tune the number of dense layers (1, 2, or 3)
    for i in range(hp.Int('dense_layers', 1, 3)):
        model.add(layers.Dense(
            units=hp.Int(f'units_{i}', min_value=32, max_value=128, step=16),
            activation='relu'))

        # Tune the dropout rate
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(rate=hp.Float(f'dropoutDense_{i}', 0.0, 0.5, step=0.1)))

    model.add(layers.Dense(10, activation='softmax'))

    # Choose an optimizer and learning rate
    optimizer = tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))

    model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])

    return model

# Defining and Running the Tuner

In [ ]:
# Step 5: Define the Tuner
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=20,
    executions_per_trial=1,
    directory='my_dir',
    project_name='cifar10_tunning'
)

# Step 6: Perform the Hyperparameter search
tuner.search(train_images, train_labels, batch_size=64, epochs=15, validation_data=(test_images, test_labels))

# Step 7: Get the best Hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Printing the best Parameters

In [ ]:
# Step 8: Print the all the best hyperparameters
print(f"Best number of convolutional layers: {best_hps.get('conv_layers')}")
print(f"Best number of deep convolutional layers: {best_hps.get('deep_conv_layers')}")
print(f"Best number of dense layers: {best_hps.get('dense_layers')}")
print(f"Best learning rate: {best_hps.get('learning_rate')}")

for i in range(best_hps.get('conv_layers')):
  print(f"Best number of filters for the convolutional layer {i+1}: {best_hps.get(f'filters_{i}')}")
  print(f"Best dropout rate for the convolutional layer {i+1}: {best_hps.get(f'dropoutConv_{i}')}")

  if best_hps.get('deep_conv_layers') > 1:
    for j in range(best_hps.get('deep_conv_layers')):
      print(f"Best number of filters for the deep convolutional layer {i+1}.{j+1}: {best_hps.get(f'filters_{i}{j}')}")

for i in range(best_hps.get('dense_layers')):
  print(f"Best number of units for the dense layer {i+1}: {best_hps.get(f'units_{i}')}")
  print(f"Best dropout rate for the dense layer {i+1}: {best_hps.get(f'dropout_{i}')}")

# Selecting and Training on Best Model

In [ ]:
# Step 9: Build the model with the best Hyperparameters and train it
model = tuner.hypermodel.build(best_hps)
early_stopping = EarlyStopping(
       monitor='val_loss',  # Metric to monitor
       patience=3,          # Number of epochs with no improvement before stopping
       restore_best_weights=True  # Restore the weights of the best epoch
   )
# Train the model
history = model.fit(train_images, train_labels, batch_size=64, epochs=100, validation_data=(test_images, test_labels))

# Plotting Accuracy and Loss

In [ ]:
# Step 10: Plotting training & validation accuracy and loss values
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.grid()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.grid()

plt.show()

# Testing the Model

In [ ]:
#Testing on the Training Data
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=4)
print(test_acc)

313/313 - 1s - 3ms/step - accuracy: 0.8172 - loss: 0.6956
0.8172000050544739
